# step 2.2 avenue cleaning
cleaning, spliting lists columns, deleting and adding columns to create the final df for the avenue website.

In [79]:
import pandas as pd
import re

# Nan and duplicates values
טיפול בנתונים החסרים בטבלה ובנתונים הכפולים. במידה של כפילות הורדה של שורות כפולות, ובמידה של נתון חסר טיפול בהתאם לצורך

In [80]:
df= pd.read_csv("avenue_prodoct_info_final_fixed_table.csv")
df.shape

(2626, 9)

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2626 entries, 0 to 2625
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num            2626 non-null   int64  
 1   brand          2626 non-null   object 
 2   product_names  2608 non-null   object 
 3   prices         2432 non-null   float64
 4   ids            2608 non-null   object 
 5   ratings        2608 non-null   float64
 6   sizes          2626 non-null   object 
 7   fabrics        2607 non-null   object 
 8   colors         2533 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 184.8+ KB


In [82]:
df[(df.ratings!= 0.0)].shape

(1972, 9)

ניתן לראות שהחוסרים נמצאים בעמודות: שם הפריט, מחיר, מספר זיהוי, דירוג, בד וצבע.

בעמודת הדירוג שהיא גם עמודת המטרה שלנו לא ניתן להכניס שרירותית ערכים בכדי למלא את החסרים כיוון שהדבר עלול לגרום להטייה משמעותית בנכונות המודל.

בעמודות השם ומס הזיהוי לא ניתן להשלים חוסרים כיוון שלכל פריט קיים ערך אינדיוידואלי.

בעמודת הבד בחרנו שלא להשלים חוסרים כיוון שלא ניתן לחזות את תרכובת הבד השונה לחלוטין בכל פריט ביגוד והכנסה של ערך שכיח\ מילוי ע"פ עוקב\קודם עלולה לגרום להטייה.

עמודת המחיר: מילוי החוסרים ע"י עוקב\קודם בטבלה אינו רלוונטי כיוון שסדר הפריטים באתר אינו ע"פ מחיר וכך גם בטבלה שהורכשה מהאתר, בנוסף השלמה על ידי שכיח העמודה גם אינו רלוונטי, נשארנו רק עם מילוי ערכים חסרים, מילוי ע"י חציון\ממוצע כל הערכים בטבלה אינו נכון כיוון שהפריטים מחולקים לפי קטגוריה ולכל קטגוריה טווח מחירים שונה, לכן הוחלט להכניס ערך זמני * עד לטיפול מאוחר יותר בנתונים אלו על סמך עמודת הקטגוריה.

עמודת הצבע: פריטים ללא צבע הינם בדרך כלל פריטים אשר מוצגים כהדפס או דוגמא מיוחדת שלא מצריכה פירוט צבע ולכן עמודות אלו ימולאו בסימן * אשר מסמל על קטגורית צבע אחר.

In [83]:
df.prices.fillna('*', inplace=True)
df.colors.fillna('*', inplace=True)

In [84]:
df_nan1=df.dropna()
df_nan1.shape

(2607, 9)

In [85]:
df_nan1[(df_nan1.ratings!= 0.0)].shape

(1953, 9)

In [86]:
df_nan1.to_csv("avenue_cleaning_temp1.csv", index=False)

In [87]:
df_nan1.duplicated().sum()

0

# Regex cleaning
ניקוי "רעשים" בעמודות בעזרת ספריית re

In [100]:
df_nan1.replace({'ids': r'Style # '}, {'ids': ''}, regex=True, inplace=True)

# deleting and adding columns
הורדת שורות לא רלוונטיות כמו שורת המספר השימשה לצורך סידורי בשלב הקודם והוספת עמודות חדשות היכולות לעזור בשלב המודל כמו עמודת הקטגוריה.

In [101]:
df_nan1.drop("num", inplace=True, axis=1)

In [102]:
Dresses=['Dress','Chemise','Maxi']
Jackets_Coats=['Hoodie','Jacket', 'Coat','Blazer','Cardigan','Cape','Poncho','Peacoat','Trench','Parka','Capri']
Jumpsuits_Rompers=['Jumpsuit', 'Romper','Set',' Jumpsuit -','Tux','Piece']
Lingerie=['Lingerie', 'Bra', 'Thong','Briefs','Bodysuit', 'Cami','Bralette','Boyshorts','G-String','Tights','Panties']
Maternity=['Maternity']
Pants=['Leggings','Legging','Trouser','Culotte', 'Skort', 'Pants', 'Biker Shorts','Short', 'Shorts','Sweatpants','Jogger' ,'Joggers','LEGGING','Jeans', 'Pant','Jegging','Jean']
Skirts=['Skirts','Skirt','Skirt-']
Sleepwear=['Sleepwear','Pajama Set','PJ Set','Mask','Slip','Tunic']
Sweaters_Knits=['Sweater','Knit']
Swimsuits_Cover_Ups=['Swimsuit','Bikini','Bottoms','Swim']
Tops=['Sweatshirt', 'Shirt', 'Tee', 'Top', 'Muscle Tank', 'Tank', 'Crop Top','Pullover','Crew','Turtleneck','Blouse','TEE','Vest' ]

category=[]
flag=0

category_names={'Dresses':Dresses,'Jackets_Coats':Jackets_Coats,'Lingerie':Lingerie,'Jumpsuits_Rompers':Jumpsuits_Rompers,
                'Maternity':Maternity,'Pants':Pants,'Skirts':Skirts,'Sleepwear':Sleepwear,
                'Sweaters_Knits':Sweaters_Knits,'Swimsuits_Cover_Ups':Swimsuits_Cover_Ups,'Tops':Tops}

for name in df_nan1.product_names:  
    for n in category_names:
        for item in category_names[n]:
            m=re.search(item,name)
            if m!=None:
                flag=1
                category.append(n)
                break
        if flag==1:
            break
    if flag==0:
        category.append(None)
    else: flag=0
            

df_nan1['category'] = category

In [91]:
df_nan2=df_nan1.dropna()
df_nan2.shape

(2594, 9)

לאחר יצירת עמודת הקטגוריה מורידים את העמודות ללא קטגוריה, גם כאן מאותן הסיבות החלטנו להוריד ולא למלא חסרים, ובנוסף לאחר בדיקה ידנית גילינו שרוב הפריטים שלא דורגו היו פריטים שאכן אינם שיכיים לקטגוריות ביגוד ולא היו רלוונטים מלכתחילה.
בנוסף רק 13 פריטים דורגו ללא קטגוריה דבר אשר לא פוגע בהתפלגות הנתונים בטבלה.

## Price column based category column
סידור הערכים החסרים בעמודת המחיר ע"פ הממוצע של כל קטגורית ביגוד. 

In [103]:
category_names=['Dresses','Jackets_Coats','Lingerie','Jumpsuits_Rompers',
                'Maternity','Pants','Skirts','Sleepwear',
                'Sweaters_Knits','Swimsuits_Cover_Ups','Tops']

category_dict={'Dresses':0, 'Jackets_Coats':1, 'Lingerie':2, 'Jumpsuits_Rompers':3, 'Maternity':4, 'Pants':5,
              'Skirts':6, 'Sleepwear':7, 'Sweaters_Knits':8, 'Swimsuits_Cover_Ups':9, 'Tops':10}

sum_list=[0]*11
num_list=[0]*11
mean_list=[0]*11

category_df=pd.DataFrame({'name':category_names, 'num':num_list ,'sum':sum_list, 'mean': mean_list})

for ind in df_nan2.index:
    if(df_nan2["prices"][ind]!='*'):
        #num per category
        category_df["num"][category_dict[df_nan2["category"][ind]]]= category_df["num"][category_dict[df_nan2["category"][ind]]]+1
        #sum per category
        category_df["sum"][category_dict[df_nan2["category"][ind]]]= category_df["sum"][category_dict[df_nan2["category"][ind]]]+df_nan2["prices"][ind]

for ind in range(0,11):
    category_df["mean"][ind]=category_df["sum"][ind]/category_df["num"][ind]
    

In [93]:
category_df  

,name,num,sum,mean
0,Dresses,605,31591.99,52.218165
1,Jackets_Coats,299,15811.40,52.880936
2,Lingerie,21,329.61,15.695714
3,Jumpsuits_Rompers,29,1663.80,57.372414
4,Maternity,0,0.00,NaN
5,Pants,426,10101.43,23.712277
6,Skirts,31,926.37,29.882903
7,Sleepwear,265,5796.51,21.873623
8,Sweaters_Knits,127,3348.75,26.368110
9,Swimsuits_Cover_Ups,3,76.00,25.333333


In [104]:
for ind in df_nan2.index:
    if(df_nan2["prices"][ind]=='*'):
        df_nan2["prices"][ind ]= category_df["mean"][category_dict[df_nan2["category"][ind]]]

In [95]:
df_nan2[(df_nan2.prices== "*")].shape

(0, 9)

# Categorization of the y label
קטגוריזציה של ערך רציף דירוג ל6 קטגוריות מרכזיות

In [105]:
labeled_rating=[]

for num, row in enumerate(df_nan2.ratings):
    
    if row<=0.5:
        labeled_rating.append(0)
        
    if row>0.5 and row<=1.5:
        labeled_rating.append(1)
        
    if row>1.5 and row<=2.5:
        labeled_rating.append(2)
        
    if row>2.5 and row<=3.5:
        labeled_rating.append(3)
    
    if row>3.5 and row<=4.5:
        labeled_rating.append(4)
        
    if row>4.5:
        labeled_rating.append(5)
        
len(labeled_rating)

2594

In [117]:
df_nan2["ratings"]=labeled_rating

In [109]:
df_nan2.to_csv("avenue_cleaning_final_clean1",index=False)

In [110]:
df_nan2

,brand,product_names,prices,ids,ratings,sizes,fabrics,colors,category
0,avenue,Ashley Pintuck Top - terracotta,34.5,217814TERRACOTTA,0,[None],"65% Viscose, 35% Polyester.",Terracotta,Tops
1,avenue,Elouise Maxi Dress - orange,39.5,217671SANDSOFTIME,0,"['14', '16', '18', '20', '22-24', '26-28', '30...",100% Viscose.,Sands Of Time,Dresses
2,avenue,Elouise Top - orange,34.5,217670SANDSOFTIME,0,"['14', '16', '18', '20', '22-24', '26-28', '30...",100% Viscose.,Sands Of Time,Tops
3,avenue,Button Knit Cardigan - cream,49.5,217660CREAM,0,"['14', '16', '18', '20', '22-24', '26-28', '30...","52% Acrylic, 26% Polyester, 22% Nylon.",Cream,Jackets_Coats
4,avenue,Button Knit Cardigan - camel,49.5,217660CAMEL,0,"['14', '16', '18', '20', '22-24', '26-28', '30...","52% Acrylic, 26% Polyester, 22% Nylon.",Camel,Jackets_Coats
...,...,...,...,...,...,...,...,...,...
2621,avenue,High Rise Jegging Mid Wash - petite,9.0,207172PTMDWS,3,"['0X', '1X', '2X', '3X', '4X', '5X']","78% Cotton, 20% Polyester, 2% Elastane.",Mid Wash,Pants
2622,avenue,High Rise Jegging Dark Wash - petite,19.99,207172PTDKWS,3,"['0X', '1X', '2X', '3X', '4X', '5X']","78% Cotton, 20% Polyester, 2% Elastane.",Dark Wash,Pants
2623,avenue,Luxe Cool Hand Slimming Pant Gray - average,23.712277,204374480GREYAVERAGE,4,"['14', '16', '18', '20', '22', '24', '26', '28...","72% Polyester, 24% Rayon, 4% Spandex. Fabric h...",Grey,Pants
2624,avenue,Wonderwall Coat - black,52.880936,138764BLACK,5,"['12', '14', '16', '18', '20', '22', '24']","Main: 95% Polyester, 4% Viscose, 1% Nylon. Lin...",Black,Jackets_Coats


# Binary split of categorical columns
עמודות השמורות כרשימה של ערכים קטגוריאלים יומרו למספר עמודות בינאריות לפי ערכים.

In [111]:
XXS_list=[0]*2594
XS_list=[0]*2594
P_list=[0]*2594
S_list=[0]*2594
M_list=[0]*2594
L_list=[0]*2594
XL_list=[0]*2594
XXL_list=[0]*2594
XXXL_list=[0]*2594
XXXXL_list=[0]*2594
X5L_list=[0]*2594
X6L_list=[0]*2594
onesize_list=[0]*2594


for num, row in enumerate(df_nan2.sizes):
    for s in range(len(row)):
        
        if re.search('One Size|\[None\]',row):
            onesize_list[num]=1

        if re.search('14|16|14-16|0X',row):
            L_list[num]=1

        if re.search('18|20|18-20|1X',row):
            XL_list[num]=1

        if re.search('22|24|22-24|2X',row):
            XXL_list[num]=1
 
        if re.search('26|28|26-28|3X',row):
            XXXL_list[num]=1

        if re.search('30|32|30-32|4X',row):
            XXXXL_list[num]=1

        if re.search('34|36|34-36|5X',row):
            X5L_list[num]=1

        if re.search('38|40|38-40|6X',row):
            X6L_list[num]=1
            
df_size=pd.DataFrame({"XXS":XXS_list, "XS":XS_list, "S":S_list, "P":P_list, "M":M_list, "L":L_list, "XL":XL_list,
                      "XXL":XXL_list, "XXXL":XXXL_list, "XXXXL":XXXXL_list, "One_size":onesize_list,"5XL":X5L_list,"6XL":X6L_list})      

df_size

,XXS,XS,S,P,M,L,XL,XXL,XXXL,XXXXL,One_size,5XL,6XL
0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,1,1,1,1,0,0,0
2,0,0,0,0,0,1,1,1,1,1,0,0,0
3,0,0,0,0,0,1,1,1,1,1,0,0,0
4,0,0,0,0,0,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589,0,0,0,0,0,1,1,1,1,1,0,1,0
2590,0,0,0,0,0,1,1,1,1,1,0,1,0
2591,0,0,0,0,0,1,1,1,1,1,0,0,0
2592,0,0,0,0,0,1,1,1,0,0,0,0,0


In [112]:
df_size.to_csv("avenue_cleaning_final_size_table.csv", index=False)

In [113]:
BLUE_list=[0]*2594
BLACK_list=[0]*2594
WHITE_list=[0]*2594
GREY_list=[0]*2594
RED_list=[0]*2594
BROWN_list=[0]*2594
ORANGE_list=[0]*2594
PINK_list=[0]*2594
GREEN_list=[0]*2594
PURPLE_list=[0]*2594
YELLOW_list=[0]*2594

OTHER_COLORS_list=[0]*2594


for num, row in enumerate(df_nan2.colors):
    
    flag=0

    if re.search('Blue|Blues|Ocean|Dark Sea|Aqua|Quiet Night|Sapphire|Navy Bouquet|Sky|Av Nvy|Light Denim|Mid Wash|Navy|Midnight|Cobalt|Giungla|Denim|Light Lagoon|Caribe|Dark Indigo|Indigo',row):
        BLUE_list[num]=1
        flag=1

    if re.search('Black|Av Blk|Darkwash|Dark|Carbon|Noir|Coal|Licorice|Jet|After Dark|Ink',row):
        BLACK_list[num]=1
        flag=1

    if re.search('Grey|Steel Blk|Silver|Charcoal|Smoke|Veer|Iron|Charcoal Marle|Chain Reaction|Oatmeal|Steel|Slate|Smoke',row):
        GREY_list[num]=1
        flag=1

    if re.search('White|Ivory|Sugar|Blanc|Nylon|Champagne|Cream|Clear|Bone|Porcelain|Fair|Eggshell',row):
        WHITE_list[num]=1
        flag=1

    if re.search('Terracotta|Chocolate|Sands Of Time|Brown|Caramel|Saddle|Coco|Creme|Almond|Frappe|Cafe|Taupe|Tan|Cashmere|Camel|Mocha|Toffe|Cinnamon|Copper|Chai|Latte|Gobi|Mandorla|Beige',row):
        BROWN_list[num]=1
        flag=1

    if re.search('Orange|Honey|Peach|Ginger|Peachy|Coral',row):
        ORANGE_list[num]=1
        flag=1

    if re.search('Pink|Bare|Tuberose|Rose|Blush|Petal|Luna Floral|Mulberry|Rouge|Magenta|Petunia',row):
        PINK_list[num]=1
        flag=1
        
    if re.search('Green|Khaki|Army|Mint|Cactus|Sage|Teal|Emerald Floral|Yucca Palm|Emerald|Jade|Pine|Olive',row):
        GREEN_list[num]=1
        flag=1
        
    if re.search('Purple|Indigo|Lilac|Sangria|Grape|Iris|Plum|Eggplant Blk|Misty Plum|Malaga|Amethyst|Purp|Fuschia Red',row):
        PURPLE_list[num]=1
        flag=1
        
    if re.search('Red|Berry|Burgundy|Bordeaux|Ruby|ruby|Scarlett',row):
        RED_list[num]=1
        flag=1
        
    if re.search('Yellow|Buttercream|Gold|Saffron|Amber|Buff',row):
        YELLOW_list[num]=1
        flag=1
        
    if flag==0:
        OTHER_COLORS_list[num]=1


            
df_color=pd.DataFrame({"Blue":BLUE_list, 'Black':BLACK_list, "Grey":GREY_list, 'White':WHITE_list, 
                      'Brown':BROWN_list, 'Orange':ORANGE_list, 'Pink':PINK_list, 'Green':GREEN_list,
                     'Purple':PURPLE_list, 'Red':RED_list, 'Yellow':YELLOW_list, 'other_colors':OTHER_COLORS_list})      

df_color

,Blue,Black,Grey,White,Brown,Orange,Pink,Green,Purple,Red,Yellow,other_colors
0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2589,1,0,0,0,0,0,0,0,0,0,0,0
2590,0,1,0,0,0,0,0,0,0,0,0,0
2591,0,0,1,0,0,0,0,0,0,0,0,0
2592,0,1,0,0,0,0,0,0,0,0,0,0


In [114]:
df_color.to_csv("avenue_cleaning_final_color_table.csv", index=False)

# Final table
חיבור כל הטבלאות השונות וניקוי של הטבלאות שלא בשימוש יותר

In [115]:
df_size= pd.read_csv("avenue_cleaning_final_size_table.csv")
df_color=pd.read_csv("avenue_cleaning_final_color_table.csv")

base_df=pd.read_csv("avenue_cleaning_final_clean1")
copy_base_df=base_df.copy()

copy_base_df.to_csv("avenue_cleaning_1_table.csv", index=False)

copy_base_df.drop(["sizes", "colors"], inplace=True, axis=1)

df = pd.concat([copy_base_df, df_size, df_color], axis=1)

df.to_csv("avenue_cleaning_final_table.csv", index=False)


In [116]:
df

,brand,product_names,prices,ids,ratings,fabrics,category,XXS,XS,S,...,Grey,White,Brown,Orange,Pink,Green,Purple,Red,Yellow,other_colors
0,avenue,Ashley Pintuck Top - terracotta,34.500000,217814TERRACOTTA,0,"65% Viscose, 35% Polyester.",Tops,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,avenue,Elouise Maxi Dress - orange,39.500000,217671SANDSOFTIME,0,100% Viscose.,Dresses,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,avenue,Elouise Top - orange,34.500000,217670SANDSOFTIME,0,100% Viscose.,Tops,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,avenue,Button Knit Cardigan - cream,49.500000,217660CREAM,0,"52% Acrylic, 26% Polyester, 22% Nylon.",Jackets_Coats,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,avenue,Button Knit Cardigan - camel,49.500000,217660CAMEL,0,"52% Acrylic, 26% Polyester, 22% Nylon.",Jackets_Coats,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589,avenue,High Rise Jegging Mid Wash - petite,9.000000,207172PTMDWS,3,"78% Cotton, 20% Polyester, 2% Elastane.",Pants,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2590,avenue,High Rise Jegging Dark Wash - petite,19.990000,207172PTDKWS,3,"78% Cotton, 20% Polyester, 2% Elastane.",Pants,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2591,avenue,Luxe Cool Hand Slimming Pant Gray - average,23.712277,204374480GREYAVERAGE,4,"72% Polyester, 24% Rayon, 4% Spandex. Fabric h...",Pants,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2592,avenue,Wonderwall Coat - black,52.880936,138764BLACK,5,"Main: 95% Polyester, 4% Viscose, 1% Nylon. Lin...",Jackets_Coats,0,0,0,...,0,0,0,0,0,0,0,0,0,0
